In [2]:
import cv2 as cv
import numpy as np

In [3]:
classes = []
with open('coco.names') as f:
    classes = [line.strip() for line in f.readlines()]

classes, len(classes)

(['person',
  'bicycle',
  'car',
  'motorbike',
  'aeroplane',
  'bus',
  'train',
  'truck',
  'boat',
  'traffic light',
  'fire hydrant',
  'stop sign',
  'parking meter',
  'bench',
  'bird',
  'cat',
  'dog',
  'horse',
  'sheep',
  'cow',
  'elephant',
  'bear',
  'zebra',
  'giraffe',
  'backpack',
  'umbrella',
  'handbag',
  'tie',
  'suitcase',
  'frisbee',
  'skis',
  'snowboard',
  'sports ball',
  'kite',
  'baseball bat',
  'baseball glove',
  'skateboard',
  'surfboard',
  'tennis racket',
  'bottle',
  'wine glass',
  'cup',
  'fork',
  'knife',
  'spoon',
  'bowl',
  'banana',
  'apple',
  'sandwich',
  'orange',
  'broccoli',
  'carrot',
  'hot dog',
  'pizza',
  'donut',
  'cake',
  'chair',
  'sofa',
  'pottedplant',
  'bed',
  'diningtable',
  'toilet',
  'tvmonitor',
  'laptop',
  'mouse',
  'remote',
  'keyboard',
  'cell phone',
  'microwave',
  'oven',
  'toaster',
  'sink',
  'refrigerator',
  'book',
  'clock',
  'vase',
  'scissors',
  'teddy bear',
  'hair

In [4]:
yolo_net = cv.dnn.readNet('yolov3.weights', 'yolov3.cfg')
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i-1] for i in yolo_net.getUnconnectedOutLayers()]

In [6]:
cap = cv.VideoCapture(0)

while cap.isOpened():
    ret, img = cap.read()

    if ret == True:
        

        # Resize the image to a specific size if needed
        # img = cv.resize(img, (2*width, 2*height))

        height, width, channels = img.shape
        blob = cv.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        yolo_net.setInput(blob)
        outs = yolo_net.forward(output_layers)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                if confidence>0.6:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)

                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = center_x - w//2
                    y = center_y - h//2

                    boxes.append([x, y, w, h])
                    confidences.append((confidence))
                    class_ids.append(class_id)

        np.random.seed(42)
        colors = np.random.uniform(0, 255, (len(classes),3))
        indexes = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        for box in indexes:
            x, y, w, h = boxes[box]
            label = str(classes[class_ids[box]])
            color = colors[class_ids[box]]
            cv.rectangle(img, (x, y), (x+w, y+h), color, 2)
            cv.putText(img, label, (x, y), cv.FONT_HERSHEY_PLAIN, 2, 0, 2)





        cv.imshow('cap', img)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

        
cap.release()
cv.destroyAllWindows()


KeyboardInterrupt: 